# Made by Aleksey Korshuk | https://github.com/AlekseyKorshuk/clip-gan


---


**Star this repository on GitHub: [link](https://github.com/AlekseyKorshuk/clip-gan)**


---



Based on *clip_music_video*

In [ ]:
#@title Download and install requirements
%cd /content
!rm /content/clip-gan
!git clone https://github.com/AlekseyKorshuk/clip-gan
%cd /content/clip-gan
!pip install -r requirements.txt
!pip install git+https://github.com/openai/CLIP.git

/content
rm: cannot remove '/content/clip-gan': No such file or directory
Cloning into 'clip-gan'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.
/content/clip-gan
  Cloning git://github.com/openai/CLIP.git to /tmp/pip-req-build-opeujlzl
  Running command git clone -q git://github.com/openai/CLIP.git /tmp/pip-req-build-opeujlzl
     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 133kB 7.3MB/s 
     |████████████████████████████████| 7.5MB 8.4MB/s 
     |████████████████████████████████| 184kB 32.7MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 3.3MB 34.0MB/s 
     |████████████████████████████████| 26.9MB 88kB/s 
     |████████████████████████████████| 276kB 36.9MB/s 
     |████████████████████████████████

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-w_admhfe
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-w_admhfe
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 12.8MB 248kB/s 
  Created wheel for clip: filename=clip-1.0-cp37-none-any.whl size=1368708 sha256=d49577499588e213691dddca3b1ac60d480646a52ac816b43649c59c27ec20fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-ihfwgmd5/wheels/79/51/d7/69f91d37121befe21d9c52332e04f592e17d1cabc7319b3e09
Successfully built clip
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [ ]:
#@title Restart runtime
import os
os.kill(os.getpid(), 9)



---


# RUN AGAIN FROM THIS PLACE


---



In [ ]:
#@title Move to /content/clip-gan
%cd /content/clip-gan

/content/clip-gan


In [ ]:
import requests 

url = 'https://v16-webapp.tiktok.com/db999498f36792035a4873f865e2af21/61ef37f5/video/tos/useast2a/tos-useast2a-ve-0068c004/add60801c39a499ca21eb478c2065461/?a=1988&br=850&bt=425&cd=0%7C0%7C1%7C0&ch=0&cr=0&cs=0&cv=1&dr=0&ds=3&er=&ft=XOQ9-3E7nz7Thae75lXq&l=2022012417354401022308016818370B81&lr=tiktok_m&mime_type=video_mp4&net=0&pl=0&qs=0&rc=anB3OWc6ZmhtOjMzNzgzM0ApaTo8Z2RoO2RkN2RmOjY3M2cyamsycjRnNWlgLS1kLzZzczMxL2ExMjAuMzI2MGJiYWA6Yw%3D%3D&vl=&vr='
r = requests.get(url, allow_redirects=True)
open('input.mp4', 'wb').write(r.content)

In [2]:
#@title Get audio from video
!ffmpeg -y -i  input.mp4 -f mp3 -ab 192000 -vn sound.mp3

ffmpeg version 4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.3)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.4.1_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-avresample --

In [1]:
#@title Generate video
epochs = 10           #@param {type:"integer"}
generator = "biggan"  #@param ["biggan", "dall-e", "stylegan"]
interpol =          30#@param {type:"integer"}


from utils                import train, Pars, create_image, create_outputfolder, init_textfile
from dall_e               import map_pixels, unmap_pixels, load_model
from stylegan             import g_synthesis
from biggan               import BigGAN
from tqdm                 import tqdm
from skimage              import img_as_ubyte
# from google.colab.patches import cv2_imshow

import tempfile
import argparse
import torch
import clip
import glob
import os
import math
import cv2
import numpy as np
import warnings

textfile    = "lyrics.txt"
audiofile   = 'sound.mp3'
lyrics      = True
sideX       = 512
sideY       = 512
video_name = "output.avi"

fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(video_name, fourcc, interpol, (sideX,sideY))


def main():

    # Automatically creates 'output' folder
    create_outputfolder()

    # Initialize Clip
    perceptor, preprocess   = clip.load('ViT-B/32')
    perceptor               = perceptor.eval()

    # Load the model
    if generator == 'biggan':
        model   = BigGAN.from_pretrained('biggan-deep-512')
        model   = model.cpu().eval()
    elif generator == 'dall-e':
        model   = load_model("https://cdn.openai.com/dall-e/decoder.pkl", "cuda")
    elif generator == 'stylegan':
        model   = g_synthesis.eval().cuda()

    # Read the textfile 
    # descs - list to append the Description and Timestamps
    descs = init_textfile(textfile)

    # list of temporary PTFiles 
    templist = []

    # Loop over the description list
    for d in tqdm(descs):

        timestamp = d[0]
        line = d[1]
        # stamps_descs_list.append((timestamp, line))

        lats = Pars(gen=generator).cpu()

         # Init Generator's latents
        if generator == 'biggan':
            par     = lats.parameters()
            lr      = 0.1#.07
        elif generator == 'stylegan':
            par     = [lats.normu]
            lr      = .01
        elif generator == 'dall-e':
            par     = [lats.normu]
            lr      = .1

        # Init optimizer
        optimizer = torch.optim.Adam(par, lr)

        # tokenize the current description with clip and encode the text
        txt = clip.tokenize(line)
        percep = perceptor.encode_text(txt.cpu()).detach().clone()

        # Training Loop
        for i in range(epochs):
            zs = train(i, model, lats, sideX, sideY, perceptor, percep, optimizer, line, txt, epochs=epochs, gen=generator)

        # save each line's last latent to a torch file temporarily
        latent_temp = tempfile.NamedTemporaryFile()
        torch.save(zs, latent_temp) #f'./output/pt_folder/{line}.pt')
        latent_temp.seek(0)
        #append it to templist so it can be accessed later
        templist.append(latent_temp)
    return templist, descs, model

def sigmoid(x):
    x = x * 2. - 1.
    return math.tanh(1.5*x/(math.sqrt(1.- math.pow(x, 2.)) + 1e-6)) / 2 + .5

def interpolate(templist, descs, model, audiofile):

    video_temp_list = []

    # interpole elements between each image
    current_lyric = ""
    print(descs)
    for idx1, pt in enumerate(descs):

        # get the next index of the descs list, 
        # if it z1_idx is out of range, break the loop
        z1_idx = idx1 + 1
        if z1_idx >= len(descs):
            break

        try:
          current_lyric = descs[idx1+1][1]
        except:
          current_lyric = ""

        print(current_lyric)

        d1 = pt[0]
        d2 = descs[z1_idx][0]
        ttime = d2 - d1

        # if it is the very first index, load the first pt temp file
        # if not assign the previous pt file (z1) to zs variable
        if idx1 == 0:
            zs = torch.load(templist[idx1])
        else:
            zs = z1
        
        # compute for the number of elements to be insert between the 2 elements
        N = round(ttime * interpol)
        # the codes below determine if the output is list (for biggan)
        # if not insert it into a list 
        if not isinstance(zs, list):
            z0 = [zs]
            z1 = [torch.load(templist[z1_idx])]
        else:
            z0 = zs
            z1 = torch.load(templist[z1_idx])
        
        # loop over the range of elements and generate the images
        image_temp_list = []
        for t in range(N):

            azs = []
            for r in zip(z0, z1):
                z_diff = r[1] - r[0] 
                inter_zs = r[0] + sigmoid(t / (N-1)) * z_diff
                azs.append(inter_zs)

            # Generate image
            with torch.no_grad():
                if generator == 'biggan':
                    img = model(azs[0], azs[1], 1).cpu().numpy()
                    img = img[0]
                elif generator == 'dall-e':
                    img = unmap_pixels(torch.sigmoid(model(azs[0])[:, :3]).cpu().float()).numpy()
                    img = img[0]
                elif generator == 'stylegan':
                    img = model(azs[0])
                image_temp = create_image(img, t, current_lyric, generator)
            
            img_last = cv2.imread(image_temp.name + ".png") 
            # cv2.putText(img_last, current_lyric, (5, int(sideY*3/4)),
            #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            global video
            video.write(img_last)

if __name__ == '__main__':
    
    warnings.filterwarnings('ignore')
    templist, descs, model = main()
    interpolate(templist, descs, model, audiofile)
    video.release()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [20:21<00:00, 174.54s/it]


[(-2208970800.0, 'start song'), (-2208970782.0, 'I guess you could say she was a sex fiend I met her in a hotel lobby dancing'), (-2208970775.0, 'Masturbating with a magazine And I could not resist when I saw little Nikki grind'), (-2208970765.0, 'The lights went out And Nikki started to grind seductively'), (-2208970758.0, "I can't tell you what she did to me But my body will never be the same"), (-2208970748.0, 'It said, "Thank you for a funky time Call me up whenever you wanna grind"'), (-2208970739.0, 'end song')]
I guess you could say she was a sex fiend I met her in a hotel lobby dancing


Lossy conversion from float32 to uint8. Range [-0.988929033279419, 0.9942270517349243]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.988929033279419, 0.9942270517349243]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.988929033279419, 0.9942270517349243]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9889289736747742, 0.9942270517349243]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.988929033279419, 0.9942270517349243]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9889287948608398, 0.9942270517349243]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9889287948608398, 0.9942269921

Lossy conversion from float32 to uint8. Range [-0.9922760128974915, 0.9930623173713684]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9923876523971558, 0.9929975867271423]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9924647808074951, 0.992866039276123]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9925035238265991, 0.9928187131881714]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9924903512001038, 0.9928284883499146]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9924140572547913, 0.9928088188171387]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.992382287979126, 0.99269306

Lossy conversion from float32 to uint8. Range [-0.9971527457237244, 0.990064263343811]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9967807531356812, 0.9900981783866882]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9962112903594971, 0.9897565245628357]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9951935410499573, 0.9887310266494751]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9948155283927917, 0.9883833527565002]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9957525134086609, 0.9883186221122742]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9956536293029785, 0.9881300

...

Lossy conversion from float32 to uint8. Range [-0.9888709187507629, 0.9881753325462341]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9888709187507629, 0.9881753325462341]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9888709187507629, 0.9881753921508789]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9888708591461182, 0.9881747364997864]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9888704419136047, 0.9881713390350342]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9888691306114197, 0.9881603717803955]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9888660907745361, 0.988134

Lossy conversion from float32 to uint8. Range [-0.9856653213500977, 0.9952409863471985]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9840148091316223, 0.995873749256134]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9841541647911072, 0.9960929751396179]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9839703440666199, 0.996699869632721]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9826939702033997, 0.9967219233512878]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9832884669303894, 0.9966441988945007]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9829683899879456, 0.99692392

Lossy conversion from float32 to uint8. Range [-0.9816908836364746, 0.9995241761207581]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.98436039686203, 0.9995842576026917]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9884687662124634, 0.9995505213737488]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9911159873008728, 0.9994434714317322]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9926729202270508, 0.9992412328720093]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9931790828704834, 0.9992300271987915]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9938523173332214, 0.99924725

Lossy conversion from float32 to uint8. Range [-0.9757502675056458, 0.9999825954437256]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9667895436286926, 0.9999746680259705]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9676685929298401, 0.9999721646308899]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9628840088844299, 0.9999672770500183]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9756180644035339, 0.9999629855155945]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9774121046066284, 0.99995356798172]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9857281446456909, 0.99995875

Lossy conversion from float32 to uint8. Range [-0.9818474054336548, 0.999994158744812]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9836441874504089, 0.9999972581863403]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9844855070114136, 0.999997079372406]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9855413436889648, 0.9999979138374329]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9865668416023254, 0.9999984502792358]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9876799583435059, 0.9999990463256836]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9887917637825012, 0.99999946

In [2]:
#@title Convert avi to mp4 and concatenate audio file
!ffmpeg -y -i output.avi output.mp4
!ffmpeg -y -i output.mp4 -i sound.mp3 -c:v copy -map 0:v:0 -map 1:a:0 -c:a aac -b:a 192k result.mp4

ffmpeg version 4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.3)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.4.1_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-avresample --

In [ ]:
#@title Download result of our generation
from google.colab import files
files.download('result.mp4') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>